In [1]:
# If you're using Google Colab and not running locally, run this cell.

## Install dependencies
!pip install wget
!apt-get install sox libsndfile1 ffmpeg
!pip install unidecode


## Install NeMo
!python -m pip install nemo_toolkit[all]

# !pip install torchtext==0.10.1
# !pip install matplotlib==3.1.3
"""
Remember to restart the runtime for the kernel to pick up any upgraded packages (e.g. matplotlib)!
Alternatively, you can uncomment the exit() below to crash and restart the kernel, in the case
that you want to use the "Run All Cells" (or similar) option.
"""
exit()

  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9672 sha256=b9d18d4e457c087f16aa9589aee25e78a760db346753c3f1baefdb03947eda76
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libsndfile1 is already the newest version (1.0.28-4ubuntu0.18.04.2).
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
The following additional packages will be installed:
  libmagic-mgc libmagic1 libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa
  libsox-fmt-base libsox3
Suggested packages:
  file libsox-fmt-all
The following NEW packages will be installed:
  libmagic-mgc libmagic1 libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa
  libsox-fmt-base libsox3 sox
0 upgraded, 8 newly installed, 0 to remove and 37 not upgraded.
Need to get 760 kB of archives.
After this operation, 6,717 kB of additional disk s

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from omegaconf import OmegaConf
import copy

In [2]:
! ls drive/MyDrive

 animals10	    farfield_ft2.nemo   farfield_ft.nemo     Практика
 checkpoints	    farfield_ft3.nemo   train_crowd9.tar
'Colab Notebooks'   farfield_ft4.nemo   train_farfield.tar


In [4]:
! cp drive/MyDrive/manifest.jsonl /content/

In [5]:
! ls

drive  manifest.jsonl  sample_data


In [3]:
# Папка, где будет размещаться датасет Golos
data_dir = '/content'

In [4]:
import glob
import os
import subprocess
import tarfile
import wget

In [5]:
# Загрузка датасета Golos
def load(golos_url, fname):
  if not os.path.exists(os.path.join(data_dir, fname)):
      golos_path = wget.download(golos_url, os.path.join(data_dir, fname))
      print(f"Dataset downloaded at: {golos_path}")
  else:
      print("Tarfile already exists.")
      golos_path = os.path.join(data_dir, fname)
  return golos_path

In [6]:
test_golos_path = load("https://sc.link/Kqr", "test.tar")

if not os.path.exists(os.path.join(data_dir, '/test/')):
    tar = tarfile.open(test_golos_path)
    tar.extractall(path=data_dir)

Dataset downloaded at: /content/test.tar


In [ ]:
train_golos_path = '/content/drive/MyDrive/train_crowd9.tar'
tar = tarfile.open(train_golos_path)
tar.extractall(path=data_dir)

In [7]:
train_golos_path = '/content/drive/MyDrive/train_farfield.tar'
tar = tarfile.open(train_golos_path)
tar.extractall(path=data_dir)

In [ ]:
! ls /content/train

farfield


In [8]:
! mv /content/drive/MyDrive/manifest.jsonl /content/train/

In [ ]:
! rm -r /content/train/farfield.jsonl

In [ ]:
! head /content/train/10hours.jsonl

In [9]:
# --- Building Manifest Files --- #
import json

# Function to build a manifest
def build_manifest(manifest_rel, manifest_abs):
    manifest_path = os.path.split(os.path.abspath(manifest_rel))[0]
    with open(manifest_rel, 'r') as fin:
        with open(manifest_abs, 'w') as fout:
            for line in fin:
                metadata = json.loads(line)
                metadata["audio_filepath"]=os.path.join(manifest_path, metadata["audio_filepath"])
                json.dump(metadata, fout)
                fout.write('\n')

def build_train_manifest(manifest_rel, manifest_abs):
    manifest_path = os.path.split(os.path.abspath(manifest_rel))[0]
    with open(manifest_rel, 'r') as fin:
        with open(manifest_abs, 'w') as fout:
            for line in fin:
                metadata = json.loads(line)
                if metadata['audio_filepath'][6] == "9":

                    metadata["audio_filepath"]=os.path.join(manifest_path, metadata["audio_filepath"])
                    json.dump(metadata, fout)
                    fout.write('\n')

def build_train_farfield_manifest(manifest_rel, manifest_abs):
    manifest_path = os.path.split(os.path.abspath(manifest_rel))[0]
    with open(manifest_rel, 'r') as fin:
        with open(manifest_abs, 'w') as fout:
            for line in fin:
                metadata = json.loads(line)
                # print(metadata['audio_filepath'][:8])
                if metadata['audio_filepath'][:8] == "farfield":
                    metadata["audio_filepath"]=os.path.join(manifest_path, metadata["audio_filepath"])
                    json.dump(metadata, fout)
                    fout.write('\n')
                


In [10]:
# Building Manifests
print("******")
train_rel = os.path.join(data_dir, 'test/farfield/manifest.jsonl')
train_abs = os.path.join(data_dir, 'test/farfield/farfield.jsonl')
if not os.path.isfile(train_abs):
  build_manifest(train_rel, train_abs)
test_manifest = train_abs
print("test_manifest", test_manifest)

******
test_manifest /content/test/farfield/farfield.jsonl


In [ ]:
# Building Manifests
print("******")
train_rel = os.path.join(data_dir, 'test/crowd/manifest.jsonl')
train_abs = os.path.join(data_dir, 'test/crowd/crowd.jsonl')
if not os.path.isfile(train_abs):
  build_manifest(train_rel, train_abs)
test_crowd_manifest = train_abs
print("test_crowd_manifest", test_crowd_manifest)

******
test_crowd_manifest /content/test/crowd/crowd.jsonl


In [ ]:
! head /content/test/crowd/crowd.jsonl

{"id": "e632f7d39c15e7edfc665b91e6f2071f", "audio_filepath": "/content/test/crowd/files/e632f7d39c15e7edfc665b91e6f2071f.wav", "text": "\u0430\u0444\u0438\u043d\u0430 \u0432\u043e\u0441\u043f\u0440\u043e\u0438\u0437\u0432\u0435\u0434\u0438 \u043c\u0443\u0437\u044b\u043a\u0443 \u0432\u043f\u0435\u0440\u0435\u043c\u0435\u0448\u043a\u0443", "duration": 4.9}
{"id": "5db5df8bb9e3b6660b2a04b34d4a355d", "audio_filepath": "/content/test/crowd/files/5db5df8bb9e3b6660b2a04b34d4a355d.wav", "text": "\u043d\u0430\u0439\u0442\u0438 \u0441\u0435\u0440\u0438\u0430\u043b \u0433\u0440\u0438\u0433\u043e\u0440\u0438\u0439 \u0440", "duration": 3.652}
{"id": "2c471aedc6979109f28cd53c58f8c4fb", "audio_filepath": "/content/test/crowd/files/2c471aedc6979109f28cd53c58f8c4fb.wav", "text": "\u043f\u0440\u044f\u043c\u043e\u0439 \u044d\u0444\u0438\u0440 \u0430\u043f\u043b \u043c\u0430\u043d\u0447\u0435\u0441\u0442\u0435\u0440 \u044e\u043d\u0430\u0439\u0442\u0435\u0434 \u0442\u043e\u0442\u0442\u0435\u043d\u0445\u044

In [ ]:
train_rel = os.path.join(data_dir, 'train/10hours.jsonl')
train_abs = os.path.join(data_dir, 'train/crowd.jsonl')
if not os.path.isfile(train_abs):
  build_train_manifest(train_rel, train_abs)
train_manifest = train_abs
print("train_manifest", train_manifest)

In [11]:
train_rel = os.path.join(data_dir, 'train/manifest.jsonl')
train_abs = os.path.join(data_dir, 'train/farfield.jsonl')
if not os.path.isfile(train_abs):
  build_train_farfield_manifest(train_rel, train_abs)
train_manifest_farfield = train_abs
print("train_manifest", train_manifest_farfield)

train_manifest /content/train/farfield.jsonl


In [ ]:
! head /content/train/farfield.jsonl

{"id": "855e01d6ba9a4aa59950d62037b87709", "audio_filepath": "/content/train/farfield/855e01d6ba9a4aa59950d62037b87709.wav", "text": "\u0430\u0444\u0438\u043d\u0430 \u0445\u043e\u0442\u0435\u043b\u0430 \u043d\u043e\u043c\u0435\u0440 \u0442\u0435\u043b\u0435\u0444\u043e\u043d\u0430 \u043f\u043e\u043c\u0435\u043d\u044f\u0442\u044c \u043a\u0430\u0440\u0442\u044b", "duration": 5.4596875}
{"id": "f6075fae0c592183f107cf781117d9b5", "audio_filepath": "/content/train/farfield/f6075fae0c592183f107cf781117d9b5.wav", "text": "\u0441\u0430\u043b\u044e\u0442 \u0445\u043e\u0447\u0443 \u043f\u043e\u0433\u043e\u0432\u043e\u0440\u0438\u0442\u044c \u0441 \u0434\u0436\u043e\u0439", "duration": 4.15125}
{"id": "2599a6cb55795b38ddd80b849d8670bc", "audio_filepath": "/content/train/farfield/2599a6cb55795b38ddd80b849d8670bc.wav", "text": "\u0434\u0436\u043e\u0439 \u043a\u0430\u043a\u0430\u044f \u0432\u0430\u043b\u044e\u0442\u0430 \u0432 \u043a\u0430\u043c\u0431\u043e\u0434\u0436\u0435", "duration": 3.9185625}

In [12]:
# NeMo's "core" package
import nemo
# NeMo's ASR collection - this collections contains complete ASR models and
# building blocks (modules) for ASR
import nemo.collections.asr as nemo_asr

[NeMo W 2021-11-30 16:46:42 optimizers:50] Apex was not found. Using the lamb or fused_adam optimizer will error out.
################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################



In [ ]:
load("https://sc.link/ZMv", "QuartzNet15x5_golos.nemo")

Dataset downloaded at: /content/QuartzNet15x5_golos.nemo


'/content/QuartzNet15x5_golos.nemo'

In [ ]:
pretrained_asr = nemo_asr.models.EncDecCTCModel.restore_from(os.path.join(data_dir, "QuartzNet15x5_golos.nemo"))

[NeMo W 2021-11-28 20:42:33 modelPT:131] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: train/golos_and_mcv.jsonl
    sample_rate: 16000
    labels:
    - ' '
    - а
    - б
    - в
    - г
    - д
    - е
    - ж
    - з
    - и
    - й
    - к
    - л
    - м
    - н
    - о
    - п
    - р
    - с
    - т
    - у
    - ф
    - х
    - ц
    - ч
    - ш
    - щ
    - ъ
    - ы
    - ь
    - э
    - ю
    - я
    batch_size: 64
    trim_silence: false
    max_duration: 20.0
    min_duration: 0.1
    num_workers: 20
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    parser: ru
    
[NeMo W 2021-11-28 20:42:33 modelPT:138] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a

[NeMo I 2021-11-28 20:42:33 features:265] PADDING: 16
[NeMo I 2021-11-28 20:42:33 features:282] STFT using torch
[NeMo I 2021-11-28 20:42:47 save_restore_connector:149] Model EncDecCTCModel was successfully restored from /content/QuartzNet15x5_golos.nemo.


In [ ]:
params_model = pretrained_asr._cfg

In [ ]:
params_model['train_ds']['manifest_filepath'] = train_manifest
params_model['train_ds']['batch_size']=16
params_model['train_ds']['parser']='base'
params_model['train_ds']

{'manifest_filepath': '/content/train/crowd.jsonl', 'sample_rate': 16000, 'labels': [' ', 'а', 'б', 'в', 'г', 'д', 'е', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я'], 'batch_size': 16, 'trim_silence': False, 'max_duration': 20.0, 'min_duration': 0.1, 'num_workers': 20, 'shuffle': True, 'is_tarred': False, 'tarred_audio_filepaths': None, 'tarred_shard_strategy': 'scatter', 'parser': 'base'}

In [ ]:
params_model['validation_ds']['manifest_filepath'] = test_manifest
params_model['validation_ds']['batch_size']=16
params_model['validation_ds']['parser']='base'
params_model['validation_ds']

{'manifest_filepath': '/content/test/farfield/farfield.jsonl', 'sample_rate': 16000, 'labels': [' ', 'а', 'б', 'в', 'г', 'д', 'е', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я'], 'batch_size': 16, 'num_workers': 20, 'shuffle': False, 'parser': 'base'}

In [ ]:
params_model['test_ds']['manifest_filepath'] = test_manifest
params_model['test_ds']['batch_size']=16
params_model['test_ds']['parser']='base'
params_model['test_ds']

{'manifest_filepath': '/content/test/farfield/farfield.jsonl', 'sample_rate': 16000, 'labels': [' ', 'а', 'б', 'в', 'г', 'д', 'е', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я'], 'batch_size': 16, 'shuffle': False, 'parser': 'base'}

In [ ]:
# Bigger batch-size = bigger throughput
# params['validation_ds']['batch_size'] = 8
# Setup the test data loader and make sure the model is on GPU
pretrained_asr.setup_test_data(test_data_config=params_model['test_ds'])
pretrained_asr.cuda()

# We will be computing Word Error Rate (WER) metric between our hypothesis and predictions.
# WER is computed as numerator/denominator.
# We'll gather all the test batches' numerators and denominators.
wer_nums = []
wer_denoms = []

# Loop over all test batches.
# Iterating over the model's `test_dataloader` will give us:
# (audio_signal, audio_signal_length, transcript_tokens, transcript_length)
# See the AudioToCharDataset for more details.
for test_batch in pretrained_asr.test_dataloader():
        test_batch = [x.cuda() for x in test_batch]
        targets = test_batch[2]
        targets_lengths = test_batch[3]        
        log_probs, encoded_len, greedy_predictions = pretrained_asr(
            input_signal=test_batch[0], input_signal_length=test_batch[1]
        )
        # Notice the model has a helper object to compute WER
        pretrained_asr._wer.update(greedy_predictions, targets, targets_lengths)
        _, wer_num, wer_denom = pretrained_asr._wer.compute()
        pretrained_asr._wer.reset()
        wer_nums.append(wer_num.detach().cpu().numpy())
        wer_denoms.append(wer_denom.detach().cpu().numpy())

        # Release tensors from GPU memory
        del test_batch, log_probs, targets, targets_lengths, encoded_len, greedy_predictions

# We need to sum all numerators and denominators first. Then divide.
print(f"WER = {sum(wer_nums)/sum(wer_denoms)}")

[NeMo I 2021-11-28 20:43:16 collections:173] Dataset loaded with 1916 files totalling 1.41 hours
[NeMo I 2021-11-28 20:43:16 collections:174] 0 files were filtered totalling 0.00 hours


[NeMo W 2021-11-28 20:43:16 patch_utils:50] torch.stft() signature has been updated for PyTorch 1.7+
    Please update PyTorch to remain compatible with later versions of NeMo.
[NeMo W 2021-11-28 20:43:17 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/nemo/collections/asr/parts/submodules/jasper.py:280: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
      ) // self.conv.stride[0] + 1
    


WER = 0.6670245764733954


In [ ]:
import pytorch_lightning as pl
trainer = pl.Trainer(gpus=1, max_epochs=50, precision=16)
pretrained_asr = nemo_asr.models.EncDecCTCModel.restore_from(os.path.join(data_dir, "QuartzNet15x5_golos.nemo"), trainer=trainer)

Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
[NeMo W 2021-11-28 20:48:59 modelPT:131] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: train/golos_and_mcv.jsonl
    sample_rate: 16000
    labels:
    - ' '
    - а
    - б
    - в
    - г
    - д
    - е
    - ж
    - з
    - и
    - й
    - к
    - л
    - м
    - н
    - о
    - п
    - р
    - с
    - т
    - у
    - ф
    - х
    - ц
    - ч
    - ш
    - щ
    - ъ
    - ы
    - ь
    - э
    - ю
    - я
    batch_size: 64
    trim_silence: false
    max_duration: 20.0
    min_duration: 0.1
    num_workers: 20
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    parser: ru
    
[NeMo W 2021-11-28 20:48:

[NeMo I 2021-11-28 20:48:59 features:265] PADDING: 16
[NeMo I 2021-11-28 20:48:59 features:282] STFT using torch
[NeMo I 2021-11-28 20:49:00 save_restore_connector:149] Model EncDecCTCModel was successfully restored from /content/QuartzNet15x5_golos.nemo.


In [ ]:
params_model = pretrained_asr._cfg
params_model['train_ds']['manifest_filepath'] = train_manifest
params_model['train_ds']['batch_size']=16
params_model['train_ds']['parser']='base'
params_model['validation_ds']['manifest_filepath'] = test_manifest
params_model['validation_ds']['batch_size']=16
params_model['validation_ds']['parser']='base'
params_model['test_ds']['manifest_filepath'] = test_manifest
params_model['test_ds']['batch_size']=16
params_model['test_ds']['parser']='base'

In [ ]:
params_model['optim']['lr'] = 0.0001

In [ ]:
import copy
new_opt = copy.deepcopy(params_model['optim'])

In [ ]:
from omegaconf import DictConfig
# Use the smaller learning rate we set before
pretrained_asr.setup_optimization(optim_config=DictConfig(new_opt))

# Point to the data we'll use for fine-tuning as the training set
pretrained_asr.setup_training_data(train_data_config=params_model['train_ds'])

# Point to the new validation data for fine-tuning
pretrained_asr.setup_validation_data(val_data_config=params_model['validation_ds'])

[NeMo W 2021-11-28 20:49:11 modelPT:471] The lightning trainer received accelerator: <pytorch_lightning.accelerators.gpu.GPUAccelerator object at 0x7efc0dd233d0>. We recommend to use 'ddp' instead.


[NeMo I 2021-11-28 20:49:11 modelPT:561] Optimizer config = Novograd (
    Parameter Group 0
        amsgrad: False
        betas: [0.9, 0.98]
        eps: 1e-08
        grad_averaging: False
        lr: 0.0001
        weight_decay: 0.001
    )


[NeMo W 2021-11-28 20:49:12 lr_scheduler:683] As `t_max_epochs` is provided/computed, it is required to pass the train dataloader in order
    to compute effective maximum number of steps.
    Scheduler will not be instantiated !


[NeMo I 2021-11-28 20:49:12 collections:173] Dataset loaded with 657 files totalling 0.54 hours
[NeMo I 2021-11-28 20:49:12 collections:174] 0 files were filtered totalling 0.00 hours


[NeMo W 2021-11-28 20:49:12 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 20 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
      cpuset_checked))
    


[NeMo I 2021-11-28 20:49:12 collections:173] Dataset loaded with 1916 files totalling 1.41 hours
[NeMo I 2021-11-28 20:49:12 collections:174] 0 files were filtered totalling 0.00 hours


In [ ]:
trainer.fit(pretrained_asr)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
[NeMo W 2021-11-28 21:13:08 modelPT:471] The lightning trainer received accelerator: <pytorch_lightning.accelerators.gpu.GPUAccelerator object at 0x7efc0dd233d0>. We recommend to use 'ddp' instead.


[NeMo I 2021-11-28 21:13:08 modelPT:561] Optimizer config = Novograd (
    Parameter Group 0
        amsgrad: False
        betas: [0.9, 0.98]
        eps: 1e-08
        grad_averaging: False
        lr: 0.0001
        weight_decay: 0.001
    )
[NeMo I 2021-11-28 21:13:08 lr_scheduler:752] Scheduler "<nemo.core.optim.lr_scheduler.CosineAnnealing object at 0x7efc0dc672d0>" 
    will be used during training (effective maximum steps = 2100) - 
    Parameters : 
    (warmup_steps: 500
    warmup_ratio: null
    min_lr: 0.0
    last_epoch: -1
    max_steps: 2100
    )



  | Name              | Type                              | Params
------------------------------------------------------------------------
0 | preprocessor      | AudioToMelSpectrogramPreprocessor | 0     
1 | encoder           | ConvASREncoder                    | 18.9 M
2 | decoder           | ConvASRDecoder                    | 34.9 K
3 | loss              | CTCLoss                           | 0     
4 | spec_augmentation | SpectrogramAugmentation           | 0     
5 | _wer              | WER                               | 0     
------------------------------------------------------------------------
18.9 M    Trainable params
0         Non-trainable params
18.9 M    Total params
37.859    Total estimated model params size (MB)
[NeMo W 2021-11-28 21:13:09 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:623: UserWarning: Checkpoint directory /content/lightning_logs/version_1/checkpoints exists and is not empty.
      rank_

Validation sanity check: 0it [00:00, ?it/s]

[NeMo W 2021-11-28 21:13:09 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 20 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
      cpuset_checked))
    
[NeMo W 2021-11-28 21:13:11 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/nemo/collections/asr/parts/submodules/jasper.py:280: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [ ]:
# Bigger batch-size = bigger throughput
# params['validation_ds']['batch_size'] = 8
# Setup the test data loader and make sure the model is on GPU
pretrained_asr.setup_test_data(test_data_config=params_model['test_ds'])
pretrained_asr.cuda()

# We will be computing Word Error Rate (WER) metric between our hypothesis and predictions.
# WER is computed as numerator/denominator.
# We'll gather all the test batches' numerators and denominators.
wer_nums = []
wer_denoms = []

# Loop over all test batches.
# Iterating over the model's `test_dataloader` will give us:
# (audio_signal, audio_signal_length, transcript_tokens, transcript_length)
# See the AudioToCharDataset for more details.
for test_batch in pretrained_asr.test_dataloader():
        test_batch = [x.cuda() for x in test_batch]
        targets = test_batch[2]
        targets_lengths = test_batch[3]        
        log_probs, encoded_len, greedy_predictions = pretrained_asr(
            input_signal=test_batch[0], input_signal_length=test_batch[1]
        )
        # Notice the model has a helper object to compute WER
        pretrained_asr._wer.update(greedy_predictions, targets, targets_lengths)
        _, wer_num, wer_denom = pretrained_asr._wer.compute()
        pretrained_asr._wer.reset()
        wer_nums.append(wer_num.detach().cpu().numpy())
        wer_denoms.append(wer_denom.detach().cpu().numpy())

        # Release tensors from GPU memory
        del test_batch, log_probs, targets, targets_lengths, encoded_len, greedy_predictions

# We need to sum all numerators and denominators first. Then divide.
print(f"WER = {sum(wer_nums)/sum(wer_denoms)}")

[NeMo I 2021-11-28 22:41:06 collections:173] Dataset loaded with 1916 files totalling 1.41 hours
[NeMo I 2021-11-28 22:41:06 collections:174] 0 files were filtered totalling 0.00 hours


RuntimeError: ignored

In [ ]:
pretrained_asr.save_to('/content/drive/MyDrive/checkpoint.nemo')

In [ ]:
! ls /content/drive/MyDrive

'Colab Notebooks'   train_crowd9.tar


In [13]:
import pytorch_lightning as pl
trainer = pl.Trainer(gpus=1, max_epochs=5, precision=16)
pretrained_asr = nemo_asr.models.EncDecCTCModel.restore_from('/content/drive/MyDrive/farfield_ft4.nemo', trainer=trainer)

Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
[NeMo W 2021-11-30 16:46:48 modelPT:131] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /content/train/farfield.jsonl
    sample_rate: 16000
    labels:
    - ' '
    - а
    - б
    - в
    - г
    - д
    - е
    - ж
    - з
    - и
    - й
    - к
    - л
    - м
    - н
    - о
    - п
    - р
    - с
    - т
    - у
    - ф
    - х
    - ц
    - ч
    - ш
    - щ
    - ъ
    - ы
    - ь
    - э
    - ю
    - я
    batch_size: 16
    trim_silence: false
    max_duration: 20.0
    min_duration: 0.1
    num_workers: 20
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    parser: base
    
[NeMo W 2021-11-30 

[NeMo I 2021-11-30 16:46:48 features:265] PADDING: 16
[NeMo I 2021-11-30 16:46:48 features:282] STFT using torch
[NeMo I 2021-11-30 16:47:12 save_restore_connector:149] Model EncDecCTCModel was successfully restored from /content/drive/MyDrive/farfield_ft4.nemo.


In [24]:
params_model = pretrained_asr._cfg

In [25]:

params_model['train_ds']['manifest_filepath'] = train_manifest_farfield 
params_model['train_ds']['batch_size']=16
params_model['train_ds']['parser']='base'
params_model['validation_ds']['manifest_filepath'] = test_manifest
params_model['validation_ds']['batch_size']=16
params_model['validation_ds']['parser']='base'
params_model['test_ds']['manifest_filepath'] = test_manifest
params_model['test_ds']['batch_size']=16
params_model['test_ds']['parser']='base'

In [31]:
params_model['optim']['lr'] = 0.00008

In [32]:
import copy
new_opt = copy.deepcopy(params_model['optim'])

In [33]:
from omegaconf import DictConfig
# Use the smaller learning rate we set before
pretrained_asr.setup_optimization(optim_config=DictConfig(new_opt))

# Point to the data we'll use for fine-tuning as the training set
pretrained_asr.setup_training_data(train_data_config=params_model['train_ds'])

# Point to the new validation data for fine-tuning
pretrained_asr.setup_validation_data(val_data_config=params_model['validation_ds'])

[NeMo W 2021-11-30 18:12:41 modelPT:471] The lightning trainer received accelerator: <pytorch_lightning.accelerators.gpu.GPUAccelerator object at 0x7f80c856c950>. We recommend to use 'ddp' instead.


[NeMo I 2021-11-30 18:12:41 modelPT:561] Optimizer config = Novograd (
    Parameter Group 0
        amsgrad: False
        betas: [0.9, 0.98]
        eps: 1e-08
        grad_averaging: False
        lr: 8e-05
        weight_decay: 0.001
    )
[NeMo I 2021-11-30 18:12:41 lr_scheduler:752] Scheduler "<nemo.core.optim.lr_scheduler.CosineAnnealing object at 0x7f7f37fe1710>" 
    will be used during training (effective maximum steps = 38750) - 
    Parameters : 
    (warmup_steps: 500
    warmup_ratio: null
    min_lr: 0.0
    last_epoch: -1
    max_steps: 38750
    )
[NeMo I 2021-11-30 18:12:45 collections:173] Dataset loaded with 123995 files totalling 132.43 hours
[NeMo I 2021-11-30 18:12:45 collections:174] 8 files were filtered totalling 0.03 hours


[NeMo W 2021-11-30 18:12:45 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 20 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
      cpuset_checked))
    


[NeMo I 2021-11-30 18:12:45 collections:173] Dataset loaded with 1916 files totalling 1.41 hours
[NeMo I 2021-11-30 18:12:45 collections:174] 0 files were filtered totalling 0.00 hours


In [ ]:
# Bigger batch-size = bigger throughput
# params['validation_ds']['batch_size'] = 8
# Setup the test data loader and make sure the model is on GPU
pretrained_asr.setup_test_data(test_data_config=params_model['test_ds'])
pretrained_asr.cuda()

# We will be computing Word Error Rate (WER) metric between our hypothesis and predictions.
# WER is computed as numerator/denominator.
# We'll gather all the test batches' numerators and denominators.
wer_nums = []
wer_denoms = []

# Loop over all test batches.
# Iterating over the model's `test_dataloader` will give us:
# (audio_signal, audio_signal_length, transcript_tokens, transcript_length)
# See the AudioToCharDataset for more details.
for test_batch in pretrained_asr.test_dataloader():
        test_batch = [x.cuda() for x in test_batch]
        targets = test_batch[2]
        targets_lengths = test_batch[3]        
        log_probs, encoded_len, greedy_predictions = pretrained_asr(
            input_signal=test_batch[0], input_signal_length=test_batch[1]
        )
        # Notice the model has a helper object to compute WER
        pretrained_asr._wer.update(greedy_predictions, targets, targets_lengths)
        _, wer_num, wer_denom = pretrained_asr._wer.compute()
        pretrained_asr._wer.reset()
        wer_nums.append(wer_num.detach().cpu().numpy())
        wer_denoms.append(wer_denom.detach().cpu().numpy())

        # Release tensors from GPU memory
        del test_batch, log_probs, targets, targets_lengths, encoded_len, greedy_predictions

# We need to sum all numerators and denominators first. Then divide.
print(f"WER = {sum(wer_nums)/sum(wer_denoms)}")

[NeMo I 2021-11-29 09:43:58 collections:173] Dataset loaded with 1916 files totalling 1.41 hours
[NeMo I 2021-11-29 09:43:58 collections:174] 0 files were filtered totalling 0.00 hours


[NeMo W 2021-11-29 09:43:58 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/nemo/collections/asr/parts/submodules/jasper.py:280: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
      ) // self.conv.stride[0] + 1
    


WER = 0.6301598663803388


In [ ]:
trainer.fit(pretrained_asr)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
[NeMo W 2021-11-30 18:12:45 modelPT:471] The lightning trainer received accelerator: <pytorch_lightning.accelerators.gpu.GPUAccelerator object at 0x7f80c856c950>. We recommend to use 'ddp' instead.


[NeMo I 2021-11-30 18:12:45 modelPT:561] Optimizer config = Novograd (
    Parameter Group 0
        amsgrad: False
        betas: [0.9, 0.98]
        eps: 1e-08
        grad_averaging: False
        lr: 8e-05
        weight_decay: 0.001
    )
[NeMo I 2021-11-30 18:12:45 lr_scheduler:752] Scheduler "<nemo.core.optim.lr_scheduler.CosineAnnealing object at 0x7f7f3ee3f250>" 
    will be used during training (effective maximum steps = 38750) - 
    Parameters : 
    (warmup_steps: 500
    warmup_ratio: null
    min_lr: 0.0
    last_epoch: -1
    max_steps: 38750
    )



  | Name              | Type                              | Params
------------------------------------------------------------------------
0 | preprocessor      | AudioToMelSpectrogramPreprocessor | 0     
1 | encoder           | ConvASREncoder                    | 18.9 M
2 | decoder           | ConvASRDecoder                    | 34.9 K
3 | loss              | CTCLoss                           | 0     
4 | spec_augmentation | SpectrogramAugmentation           | 0     
5 | _wer              | WER                               | 0     
------------------------------------------------------------------------
18.9 M    Trainable params
0         Non-trainable params
18.9 M    Total params
37.859    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

[NeMo W 2021-11-30 18:12:47 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 20 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
      cpuset_checked))
    
[NeMo W 2021-11-30 18:12:48 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/nemo/collections/asr/parts/submodules/jasper.py:280: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b

Training: 0it [00:00, ?it/s]

In [35]:
params_model['test_ds']['manifest_filepath'] = test_manifest
params_model['test_ds']['batch_size']=16
params_model['test_ds']['parser']='base'
# Bigger batch-size = bigger throughput
# params['validation_ds']['batch_size'] = 8
# Setup the test data loader and make sure the model is on GPU
pretrained_asr.setup_test_data(test_data_config=params_model['test_ds'])
pretrained_asr.cuda()

# We will be computing Word Error Rate (WER) metric between our hypothesis and predictions.
# WER is computed as numerator/denominator.
# We'll gather all the test batches' numerators and denominators.
wer_nums = []
wer_denoms = []

# Loop over all test batches.
# Iterating over the model's `test_dataloader` will give us:
# (audio_signal, audio_signal_length, transcript_tokens, transcript_length)
# See the AudioToCharDataset for more details.
for test_batch in pretrained_asr.test_dataloader():
        test_batch = [x.cuda() for x in test_batch]
        targets = test_batch[2]
        targets_lengths = test_batch[3]        
        log_probs, encoded_len, greedy_predictions = pretrained_asr(
            input_signal=test_batch[0], input_signal_length=test_batch[1]
        )
        # Notice the model has a helper object to compute WER
        pretrained_asr._wer.update(greedy_predictions, targets, targets_lengths)
        _, wer_num, wer_denom = pretrained_asr._wer.compute()
        pretrained_asr._wer.reset()
        wer_nums.append(wer_num.detach().cpu().numpy())
        wer_denoms.append(wer_denom.detach().cpu().numpy())

        # Release tensors from GPU memory
        del test_batch, log_probs, targets, targets_lengths, encoded_len, greedy_predictions

# We need to sum all numerators and denominators first. Then divide.
print(f"WER = {sum(wer_nums)/sum(wer_denoms)}")

[NeMo I 2021-11-30 16:10:32 collections:173] Dataset loaded with 1916 files totalling 1.41 hours
[NeMo I 2021-11-30 16:10:32 collections:174] 0 files were filtered totalling 0.00 hours


[NeMo W 2021-11-30 16:10:32 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/nemo/collections/asr/parts/submodules/jasper.py:280: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
      ) // self.conv.stride[0] + 1
    


WER = 0.5275590551181102


In [ ]:
pretrained_asr.save_to('/content/drive/MyDrive/farfield_ft2.nemo')

WER before: 0.667

current WER: 0.5276